# Initialize data frame

In [1]:
import pandas as pd

In [2]:
pmids = sorted(list(pd.read_csv("../brain/coordinates.csv")["PMID"]))
len(pmids)

18155

In [3]:
df = pd.DataFrame({"PMID": pmids})
df["N"] = ""
df["SOURCE"] = ""

In [4]:
found = []

# Add data from BrainMap

In [5]:
bm_coords = open("brainmap_coords_180803.txt", "r").read().split("\n\n")
bm_coords[:3]

['// Anderson N D, 2000: FA, Encoding > Retrieval in Young\n// Subjects=12\n1.7\t-6.74\t45.36\n-52.9\t41.22\t10.17\n-44.06\t18.68\t21.21\n-46.76\t40.87\t-16.8\n-50.8\t-48.9\t-12.33\n-31.8\t9.07\t-36.32\n59.43\t17.38\t1.65\n57.49\t5.86\t16.27\n65.88\t-2.57\t-5.45\n56.92\t5.96\t-28.56\n20.78\t-59.75\t5.45\n40.13\t-95.37\t-9.28\n13.86\t-21.43\t-20.63\n-35.58\t45.95\t13.89\n-52.67\t-35.77\t13.3\n-41.76\t-61.26\t15.63\n-57.43\t-10.63\t-16\n-42.44\t-18.2\t-28.96\n-52.43\t-38.37\t31.48\n-26.68\t-94.36\t5.21\n46.68\t-68.58\t1.4\n39.61\t-18.33\t-34.82\n38.47\t-32.55\t42.8',
 '// Anderson N D, 2000: FA, Retrieval > Encoding in Young\n// Subjects=12\n-27.39\t60.09\t-19.03\n-16.33\t53.28\t-.62\n-11.43\t-53.94\t23.36\n-9.27\t-97.72\t14.21\n42.09\t47.52\t3.44\n7.67\t52.9\t16.94\n16.12\t-62.25\t-21.11\n1.29\t-18.57\t10.68\n-14.5\t33.36\t-30.05\n-3.11\t44.35\t17.97\n-37.97\t15.8\t-9.98\n-6.93\t-70.11\t33.85\n-5.31\t-78.52\t-10.17\n-20.64\t5.22\t-9.23\n22.28\t22.86\t-29.64\n15.95\t56.82\t-10.48\n35.45\

In [6]:
bm_df = pd.read_csv("brainmap_metadata_180809.csv", encoding="latin-1")
bm_df.head(3)

,BRAINMAP_ID,PMID,KEY,1st_AUTHOR,AUTHORS,YEAR,TITLE,JOURNAL,VOLUME,MONTH,PAGES,BEHAVIORAL_DOMAIN,EXPERIMENT,DESCRIPTION,ABSTRACT_URL,NUM_COORDINATES,DOI
0,16050115,12953303,"Aasen I, 2003",Aasen I,Aasen I|Hashimoto Y|Sakai K L,2003,Brain activations during conscious self-monito...,Human Brain Mapping,20,Sep,22-28,"['Cognition.Language.Speech', 'Cognition.Langu...",['Brain Regions Selectively Associated with th...,NaN,http://www.ncbi.nlm.nih.gov/entrez/query.fcgi?...,19,10.1002/hbm.10119
1,7110338,15858160,"Abe N, 2006",Abe N,Abe N|Suzuki M|Tsukiura T|Mori E|Yamaguchi K|I...,2006,Dissociable roles of prefrontal and anterior c...,Cerebral Cortex,16,Feb,192-199,['Cognition.Social Cognition'],"['(Lie, Old - Truth, Old) + (Lie, New - Truth,...",Subjects underwent 4 conditions in which they ...,http://www.ncbi.nlm.nih.gov/entrez/query.fcgi?...,4,10.1093/cercor/bhi097
2,7110339,17280517,"Abe N, 2007",Abe N,Abe N|Suzuki M|Mori E|Itoh M|Fujii T,2007,Deceiving others: Distinct neural responses of...,Journal of Cognitive Neuroscience,19,Feb,287-295,"['Cognition.Social Cognition,Emotion.Negative....",['Main Effect of Falsifying the Truthful Respo...,Subjects underwent 4 conditions in which they ...,http://www.ncbi.nlm.nih.gov/entrez/query.fcgi?...,12,10.1162/jocn.2007.19.2.287


In [7]:
bm_n = {}
for study in bm_coords:
    key = study.split(":")[0].replace("// ", "")
    if "Subjects=" in study:
        n = float(study.split("Subjects=")[1].split("\n")[0])
        if key in bm_n.keys():
            bm_n[key].append(n)
        else:
            bm_n[key] = [n]

In [8]:
bm2pmid = {row["KEY"]: row["PMID"] for i, row in bm_df.iterrows()}

In [9]:
for key, n in bm_n.items():
    mean_n = sum(n) / len(n)
    if key in bm2pmid.keys():
        pmid = bm2pmid[key]
        df.loc[df["PMID"] == pmid, "N"] = mean_n
        df.loc[df["PMID"] == pmid, "SOURCE"] = "BrainMap"
        found.append(pmid)
    else:
        print(key)

Shaywitz B A, 1995
Bookheimer S Y, 1995
Awh E, 1996
de Araujo I E, 2003b
De Bleser R, 2003
Bartels A, 2000b
De Nil L F, 2008
Gilger J W, 2013
Morita T, 2016
Ishitobi M, 2011
Lundström J N, 2008
Kisely M, 2001
Herrington J D, 2007
Kucian K, 2005
Owens T E, 2010


# Add data collected manually

In [10]:
demo = pd.read_csv("ehb_nrsa.csv")
demo = demo.dropna(subset=["N"])
demo.head(3)

,PMID,MODALITY,MODALITY_CATEGORY,AGE_MIN,AGE_MAX,AGE_MEAN,AGE_SD,N_FEMALE,N_MALE,N
0,27894900,fMRI,Functional MRI,18.0,28.0,21.9,NaN,11.0,6.0,17.0
1,22922525,fMRI,Functional MRI,NaN,NaN,33.2,6.10,12.0,15.0,27.0
2,17336346,fMRI,Functional MRI,NaN,NaN,25.7,10.15,NaN,NaN,20.0


In [11]:
for i, row in demo.iterrows():
    pmid = row["PMID"]
    if pmid not in found:
        df.loc[df["PMID"] == pmid, "N"] = row["N"]
        df.loc[df["PMID"] == pmid, "SOURCE"] = "EHB"
        found.append(pmid)

In [12]:
ehb = pd.read_csv("ehb_subjects.csv")
ehb = ehb.loc[ehb["SOURCE"] == "EHB"]
ehb.head(3)

,PMID,N,SOURCE
112,9065511,4.0,EHB
118,9114263,3.0,EHB
131,9185551,15.5,EHB


In [13]:
for i, row in ehb.iterrows():
    pmid = row["PMID"]
    if pmid not in found:
        df.loc[df["PMID"] == pmid, "N"] = row["N"]
        df.loc[df["PMID"] == pmid, "SOURCE"] = "EHB"
        found.append(pmid)

# Add data by string matching

In [14]:
import os, re
from word2number import w2n

In [15]:
primary_patterns = [r"(?<=[nN]\s\=\s)\d+",
                    r"(?<=[nN]\=)\d+",
                    r"\w+(?=\s+healthy)",
                    r"\w+(?=\s+normal)",
                    r"\w+(?=\s+human)",
                    r"\w+(?=\s+young)",
                    r"\w+(?=\s+righthanded)", 
                    r"\w+(?=\s+matched)", 
                    r"\w+(?=\s+age)",
                    r"\w+(?=\s+sex)",
                    r"\w+(?=\s+gender)",
                    r"\w+(?=\s+english)",
                    r"\w+(?=\s+native)",
                    r"\w+(?=\s+demographically)",
                    r"\w+(?=\s+neurologically)",
                    r"\w+(?=\s+congenitally)",
                    r"\w+(?=\s+typically)",
                    r"\w+(?=\s+neurotypical)",
                    r"\w+(?=\s+subject)",
                    r"\w+(?=\s+participant)", 
                    r"\w+(?=\s+volunteer)",
                    r"\w+(?=\s+observer)",
                    r"\w+(?=\s+perceiver)",
                    r"\w+(?=\s+listener)",
                    r"\w+(?=\s+blind)",
                    r"\w+(?=\s+deaf)",
                    r"\w+(?=\s+individual)",
                    r"\w+(?=\s+person)",
                    r"\w+(?=\s+people)",
                    r"\w+(?=\s+adult)",
                    r"\w+(?=\s+adolescent)",
                    r"\w+(?=\s+child)",
                    r"\w+(?=\s+youth)",
                    r"\w+(?=\s+student)",
                    r"\w+(?=\s+undergraduate)",
                    r"\w+(?=\s+graduate)",
                    r"\w+(?=\s+college)",
                    r"\w+(?=\s+university)",
                    r"\w+(?=\s+patient)",
                    r"\w+(?=\s+medicat)",
                    r"\w+(?=\s+outpatient)",
                    r"\w+(?=\s+veteran)"]

secondary_patterns = [r"\w+(?=\s+female)",
                      r"\w+(?=\s+male)",
                      r"\w+(?=\s+women)",
                      r"\w+(?=\s+men)",
                      r"\w+(?=\s+girl)",
                      r"\w+(?=\s+boy)",
                      r"\w+(?=\s+old)",
                      r"\w+(?=\s+elderly)",
                      r"\w+(?=\s+control)",
                      r"\w+(?=\s+hc)",
                      r"\w+(?=\s+comparison)",
                      r"\w+(?=\s+sighted)",
                      r"\w+(?=\s+hearing)"]

In [16]:
nums = ["twenty", 
        "thirty", 
        "forty", 
        "fifty", 
        "sixty", 
        "seventy", 
        "eighty", 
        "ninety"]

In [17]:
def str2float(n):
    return float(w2n.word_to_num(n))

def matches_to_n(matches, ns, min_n, max_n):
    for match in matches:
        n = match.strip()
        if n.isdigit():
            n = float(n)
            if n >= min_n and n <= max_n:
                ns.append(n)
        else: 
            try:
                n = str2float(n)
                if n >= min_n and n <= max_n:
                    ns.append(n)
            except:
                for num in nums:
                    n = n.replace(num, num + " ")
                try:
                    n = str2float(n)
                    if n >= min_n and n <= max_n:
                        ns.append(n)
                except:
                    pass
    return ns

def extract_n_from_file(primary_patterns, secondary_patterns, 
                        text, pmid2n, pmid, min_n, max_n):
    ns, n = [], 0
    text = text.lower().replace("-", "").replace(",", "")
    for pattern in primary_patterns:
        matches = re.findall(pattern, text)
        ns = matches_to_n(matches, ns, min_n, max_n)
        if len(ns) > 1:
            break
    if len(ns) == 0:
        for pattern in secondary_patterns:
            matches = re.findall(pattern, text)
            ns = matches_to_n(matches, ns, min_n, max_n)
            if len(ns) > 1:
                break
    if len(set(ns)) == 1:
        n = ns[0]
    return n

def extract_n_from_corpus(primary_patterns, secondary_patterns, 
                          path, source_name, pmids, found, df, 
                          min_n=1, max_n=1000):
    pmid2n = {}
    for i, pmid in enumerate(pmids):
        if pmid not in found:
            file = "{}/{}.txt".format(path, pmid)
            if os.path.exists(file):
                text = open(file, "r").read()
                n = extract_n_from_file(primary_patterns, secondary_patterns, 
                                        text, pmid2n, pmid, min_n, max_n)
                if n > 0:
                    pmid2n[pmid] = n
        if i % 1000 == 0:
            print("Processed {} documents".format(i))
    for pmid, n in pmid2n.items():
        df.loc[df["PMID"] == pmid, "N"] = n
        df.loc[df["PMID"] == pmid, "SOURCE"] = source_name             
        found.append(pmid)
    print("Found data for {} documents".format(len(pmid2n.keys())))

In [18]:
extract_n_from_corpus(primary_patterns, secondary_patterns,
                      "../../../pubmed/abstracts", "Abstract", pmids, found, df)

Processed 0 documents
Processed 1000 documents
Processed 2000 documents
Processed 3000 documents
Processed 4000 documents
Processed 5000 documents
Processed 6000 documents
Processed 7000 documents
Processed 8000 documents
Processed 9000 documents
Processed 10000 documents
Processed 11000 documents
Processed 12000 documents
Processed 13000 documents
Processed 14000 documents
Processed 15000 documents
Processed 16000 documents
Processed 17000 documents
Processed 18000 documents
Found data for 3957 documents


In [19]:
extract_n_from_corpus(primary_patterns, secondary_patterns, 
                      "../../../cogneuro/texts/raw", "FullText", pmids, found, df)

Processed 0 documents
Processed 1000 documents
Processed 2000 documents
Processed 3000 documents
Processed 4000 documents
Processed 5000 documents
Processed 6000 documents
Processed 7000 documents
Processed 8000 documents
Processed 9000 documents
Processed 10000 documents
Processed 11000 documents
Processed 12000 documents
Processed 13000 documents
Processed 14000 documents
Processed 15000 documents
Processed 16000 documents
Processed 17000 documents
Processed 18000 documents
Found data for 950 documents


# Export worksheet for manual collection

In [20]:
sum(df["SOURCE"] == "")

0

# Export the data

In [21]:
df.to_csv("subjects.csv", index=None)

In [22]:
pmid2subjects = {df["PMID"][i]: df["N"][i] for i in range(len(df))}

In [23]:
meta = pd.read_csv("metadata.csv", encoding="latin-1")
meta["N_SUBJECTS"] = [pmid2subjects[pmid] for pmid in meta["PMID"]]

In [24]:
meta.to_csv("metadata.csv", index=None)

# Number from each source

In [25]:
for source in ["FullText", "Abstract", "EHB", "BrainMap"]:
    print("{:9s} {}".format(source, len(df.loc[df["SOURCE"] == source])))

FullText  950
Abstract  3957
EHB       9904
BrainMap  3344
